# 🧪 Codelab: Shor's Algorithm - Quantum Factoring

| Metadata | Value |
|----------|-------|
| **Algorithm** | Shor's Factoring Algorithm |
| **Difficulty** | 🔴 Advanced |
| **Time** | 120-150 minutes |
| **Prerequisites** | QPE, QFT, Modular Arithmetic |
| **Qiskit Version** | 2.x |

---

## Learning Objectives

By the end of this codelab, you will be able to:

1. ✅ Understand the factoring → period finding reduction
2. ✅ Implement modular exponentiation circuits
3. ✅ Build the complete order-finding quantum circuit
4. ✅ Extract periods using continued fractions
5. ✅ Factor N=15 with Shor's algorithm

---

**⚠️ Note**: Full-scale Shor's algorithm requires fault-tolerant quantum computers. This codelab demonstrates the principles using small examples that can run on simulators.

## Section 1: Environment Setup & Version Check

In [ ]:
# Required imports
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Optional, Tuple, Dict
from fractions import Fraction
from math import gcd, log2, ceil
import random

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector, Operator
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit.circuit.library import QFT

# Version check - Qiskit 2.x required
import qiskit
version = qiskit.__version__
major_version = int(version.split('.')[0])
assert major_version >= 1, f"Qiskit 2.x required, found {version}"
print(f"✓ Qiskit version: {version}")
print(f"✓ NumPy version: {np.__version__}")
print("✓ All imports successful!")

## Section 2: Theory Recap

### The Reduction: Factoring → Period Finding

**Shor's Key Insight**: Factoring $N$ reduces to finding the period of:
$$f(x) = a^x \mod N$$

where $a$ is a random number coprime to $N$.

### Why This Works

If $r$ is the smallest positive integer where $a^r \equiv 1 \mod N$ (the **order** of $a$):

1. $a^r - 1 \equiv 0 \mod N$
2. $(a^{r/2} - 1)(a^{r/2} + 1) \equiv 0 \mod N$ (if $r$ even)
3. $\gcd(a^{r/2} - 1, N)$ and $\gcd(a^{r/2} + 1, N)$ likely give non-trivial factors!

### Quantum Speedup

- **Classical**: Finding period takes exponential time
- **Quantum**: QPE finds period in polynomial time $O(n^3)$

### Example: N = 15, a = 7

| x | 7^x mod 15 |
|---|------------|
| 0 | 1 |
| 1 | 7 |
| 2 | 4 |
| 3 | 13 |
| 4 | 1 ← period! |

Period $r = 4$, so:
- $7^2 = 49 \equiv 4 \mod 15$
- $\gcd(4-1, 15) = \gcd(3, 15) = 3$ ✓
- $\gcd(4+1, 15) = \gcd(5, 15) = 5$ ✓

## Section 3: Classical Components

In [ ]:
def classical_preprocessing(N: int) -> Tuple[bool, Optional[int]]:
    """
    Classical preprocessing before quantum algorithm.
    
    Returns:
        (is_simple_case, factor_if_found)
    """
    # Check if even
    if N % 2 == 0:
        return True, 2
    
    # Check if prime power: N = a^b
    for b in range(2, int(np.log2(N)) + 1):
        a = round(N ** (1/b))
        if a ** b == N:
            return True, a
    
    return False, None


def pick_random_a(N: int) -> Tuple[int, Optional[int]]:
    """
    Pick random a coprime to N.
    
    Returns:
        (a, factor_if_lucky)
    """
    a = random.randint(2, N - 2)
    d = gcd(a, N)
    
    if d > 1:
        # Lucky! We found a factor without quantum
        return a, d
    
    return a, None


def continued_fractions(numerator: int, denominator: int, max_denom: int) -> List[Tuple[int, int]]:
    """
    Compute convergents of continued fraction expansion.
    
    Args:
        numerator, denominator: The fraction to expand
        max_denom: Maximum denominator to consider
    
    Returns:
        List of convergent fractions (numerator, denominator)
    """
    convergents = []
    
    # Use Fraction for exact arithmetic
    frac = Fraction(numerator, denominator)
    
    # Compute continued fraction coefficients
    x = frac
    coeffs = []
    while True:
        a = int(x)
        coeffs.append(a)
        frac_part = x - a
        if frac_part == 0:
            break
        x = 1 / frac_part
    
    # Compute convergents
    # h[-1] = 1, h[0] = a_0
    # k[-1] = 0, k[0] = 1
    h_prev, h_curr = 1, coeffs[0]
    k_prev, k_curr = 0, 1
    
    convergents.append((h_curr, k_curr))
    
    for i in range(1, len(coeffs)):
        a = coeffs[i]
        h_new = a * h_curr + h_prev
        k_new = a * k_curr + k_prev
        
        if k_new > max_denom:
            break
        
        convergents.append((h_new, k_new))
        h_prev, h_curr = h_curr, h_new
        k_prev, k_curr = k_curr, k_new
    
    return convergents


# Test continued fractions
print("Continued Fractions Test")
print("=" * 50)

# 7/16 should give convergents approaching s/r
convergents = continued_fractions(7, 16, 20)
print(f"Continued fractions of 7/16:")
for num, denom in convergents:
    print(f"  {num}/{denom} = {num/denom:.6f}")

In [ ]:
def extract_period_candidates(measurement: int, n_counting: int, N: int) -> List[int]:
    """
    Extract period candidates from QPE measurement.
    
    Args:
        measurement: The measured integer from counting register
        n_counting: Number of counting qubits
        N: The number being factored
    
    Returns:
        List of possible period values
    """
    denominator = 2 ** n_counting
    
    # Get convergents
    convergents = continued_fractions(measurement, denominator, N)
    
    # Extract unique denominators as period candidates
    candidates = []
    for _, denom in convergents:
        if denom > 0 and denom not in candidates:
            candidates.append(denom)
    
    return candidates


def verify_period(a: int, r: int, N: int) -> bool:
    """
    Verify if r is the period of a^x mod N.
    """
    return pow(a, r, N) == 1


def factors_from_period(a: int, r: int, N: int) -> Tuple[Optional[int], Optional[int]]:
    """
    Extract factors from period.
    
    Returns:
        (factor1, factor2) or (None, None) if failed
    """
    if r % 2 != 0:
        return None, None  # Odd period
    
    x = pow(a, r // 2, N)
    
    if x == N - 1:  # x ≡ -1 mod N
        return None, None  # Trivial case
    
    factor1 = gcd(x - 1, N)
    factor2 = gcd(x + 1, N)
    
    # Check if non-trivial
    if factor1 in [1, N] and factor2 in [1, N]:
        return None, None
    
    if factor1 not in [1, N]:
        factor2 = N // factor1
    else:
        factor1 = N // factor2
    
    return factor1, factor2


# Test with N=15, a=7
print("\nClassical Period Verification Test")
print("=" * 50)
N = 15
a = 7

print(f"N = {N}, a = {a}")
print(f"\nPowers of {a} mod {N}:")
for x in range(10):
    print(f"  {a}^{x} mod {N} = {pow(a, x, N)}")

# Verify period r=4
r = 4
print(f"\nIs r={r} the period? {verify_period(a, r, N)}")

# Extract factors
f1, f2 = factors_from_period(a, r, N)
print(f"Factors: {f1} × {f2} = {f1 * f2}")

## Section 4: Modular Exponentiation Circuit

The quantum part of Shor's algorithm requires implementing the unitary:
$$U_a|y\rangle = |ay \mod N\rangle$$

For the textbook example N=15, we can hard-code efficient circuits.

In [ ]:
def controlled_modular_mult_15(a: int, power: int) -> QuantumCircuit:
    """
    Create controlled-U^(2^power) for modular multiplication mod 15.
    
    U|y⟩ = |a*y mod 15⟩ for y < 15
    
    This is a simplified version using the known structure of mod 15.
    
    Args:
        a: Base for modular exponentiation (coprime to 15)
        power: Exponent j where we compute U^(2^j)
    
    Returns:
        Controlled circuit implementing C-U^(2^power)
    """
    # Compute a^(2^power) mod 15
    a_pow = pow(a, 2**power, 15)
    
    # For mod 15, we can represent numbers with 4 qubits
    # |y⟩ = |y3 y2 y1 y0⟩ where y = 8*y3 + 4*y2 + 2*y1 + y0
    
    qc = QuantumCircuit(4, name=f'U_{a}^{{{2**power}}}')
    
    # Implement specific permutations based on a_pow
    # These are hand-optimized for common values
    
    if a_pow == 1:
        # Identity - do nothing
        pass
    
    elif a_pow == 2:
        # Multiply by 2 mod 15: {1→2, 2→4, 4→8, 7→14, 8→1, 11→7, 13→11, 14→13}
        qc.swap(0, 1)
        qc.swap(1, 2)
        qc.swap(2, 3)
    
    elif a_pow == 4:
        # Multiply by 4 mod 15: {1→4, 2→8, 4→1, 7→13, 8→2, 11→14, 13→7, 14→11}
        qc.swap(0, 2)
        qc.swap(1, 3)
    
    elif a_pow == 7:
        # Multiply by 7 mod 15: {1→7, 2→14, 4→13, 7→4, 8→11, 11→2, 13→1, 14→8}
        qc.swap(0, 1)
        qc.swap(1, 2)
        qc.swap(2, 3)
        qc.x(0)
        qc.x(1)
        qc.x(2)
        qc.x(3)
    
    elif a_pow == 8:
        # Multiply by 8 mod 15: {1→8, 2→1, 4→2, 7→11, 8→4, 11→13, 13→14, 14→7}
        qc.swap(0, 3)
        qc.swap(1, 2)
    
    elif a_pow == 11:
        # Multiply by 11 mod 15
        qc.swap(0, 3)
        qc.swap(1, 2)
        qc.x(0)
        qc.x(1)
        qc.x(2)
        qc.x(3)
    
    elif a_pow == 13:
        # Multiply by 13 mod 15
        qc.swap(0, 2)
        qc.swap(1, 3)
        qc.x(0)
        qc.x(1)
        qc.x(2)
        qc.x(3)
    
    elif a_pow == 14:
        # Multiply by 14 mod 15 (same as -1)
        qc.x(0)
        qc.x(1)
        qc.x(2)
        qc.x(3)
    
    else:
        raise ValueError(f"a^(2^power) mod 15 = {a_pow} not implemented")
    
    return qc


# Test the modular multiplication circuit
print("Modular Multiplication Test: U_7 (multiply by 7 mod 15)")
print("=" * 50)

# Create circuit for multiplying by 7 mod 15
u7 = controlled_modular_mult_15(7, 0)  # 7^(2^0) = 7
print("Circuit for U_7:")
print(u7.draw())

# Verify by simulation
print("\nVerification:")
for y in [1, 2, 4, 7, 8]:
    # Create state |y⟩
    qc = QuantumCircuit(4)
    binary = format(y, '04b')
    for i, bit in enumerate(reversed(binary)):
        if bit == '1':
            qc.x(i)
    qc.compose(u7, inplace=True)
    
    # Get output state
    state = Statevector(qc)
    probs = state.probabilities_dict()
    output = int(max(probs, key=probs.get)[::-1], 2)
    
    expected = (7 * y) % 15
    status = "✓" if output == expected else "✗"
    print(f"  |{y}⟩ → |{output}⟩ (expected: |{expected}⟩) {status}")

## Section 5: Building the Complete Shor Circuit

In [ ]:
def shor_circuit(a: int, N: int, n_count: int) -> QuantumCircuit:
    """
    Build Shor's algorithm circuit for factoring N.
    
    This implementation is specific to N=15.
    
    Args:
        a: Random base coprime to N
        N: Number to factor (must be 15 for this implementation)
        n_count: Number of counting qubits
    
    Returns:
        Complete Shor's algorithm circuit
    """
    assert N == 15, "This implementation only supports N=15"
    assert gcd(a, N) == 1, f"a={a} must be coprime to N={N}"
    
    # Registers
    # Counting register: n_count qubits for phase estimation
    # Work register: 4 qubits to represent numbers mod 15
    count_reg = QuantumRegister(n_count, 'count')
    work_reg = QuantumRegister(4, 'work')
    classical = ClassicalRegister(n_count, 'result')
    
    qc = QuantumCircuit(count_reg, work_reg, classical)
    
    # Step 1: Initialize work register to |1⟩
    qc.x(work_reg[0])  # |0001⟩ = |1⟩
    
    # Step 2: Hadamard on counting qubits
    qc.h(count_reg)
    
    qc.barrier(label='init')
    
    # Step 3: Controlled modular exponentiation
    # count[n-1] controls U^1
    # count[n-2] controls U^2
    # ...
    # count[0] controls U^(2^(n-1))
    
    for j in range(n_count):
        # Which power of 2?
        power = n_count - 1 - j
        
        # Get the modular multiplication circuit
        u_circuit = controlled_modular_mult_15(a, power)
        
        # Make it controlled by count_reg[j]
        controlled_u = u_circuit.control(1)
        
        # Apply: control is count_reg[j], targets are work_reg
        qc.compose(controlled_u, 
                   [count_reg[j]] + list(work_reg),
                   inplace=True)
    
    qc.barrier(label='CU')
    
    # Step 4: Inverse QFT on counting register
    inv_qft = QFT(n_count, inverse=True, do_swaps=True)
    qc.compose(inv_qft, count_reg, inplace=True)
    
    qc.barrier(label='QFT†')
    
    # Step 5: Measure counting register
    qc.measure(count_reg, classical)
    
    return qc


# Build Shor's circuit for N=15, a=7
N = 15
a = 7
n_count = 8  # 8 counting qubits for good precision

shor_qc = shor_circuit(a, N, n_count)

print(f"Shor's Algorithm Circuit for N={N}, a={a}")
print(f"Counting qubits: {n_count}")
print(f"Total qubits: {shor_qc.num_qubits}")
print(f"\nCircuit (abbreviated):")
print(shor_qc.draw(fold=80))

## Section 5.5: State Evolution Analysis - Period-to-Factor Conversion

### 5.5.1 The Core Identity and "Period-to-Factor Conversion" Rule

Shor's algorithm reduces factoring to period finding using this key insight:

**The Period Finding Connection:**

For $f(x) = a^x \mod N$, if $r$ is the smallest positive integer such that $a^r \equiv 1 \pmod{N}$, then:

$$a^r - 1 \equiv 0 \pmod{N}$$

**The Factorization Trick:**

If $r$ is even:
$$(a^{r/2})^2 - 1 \equiv 0 \pmod{N}$$
$$(a^{r/2} - 1)(a^{r/2} + 1) \equiv 0 \pmod{N}$$

**The Period-to-Factor Conversion Rule:**
> If $a^{r/2} \not\equiv \pm 1 \pmod{N}$, then $\gcd(a^{r/2} \pm 1, N)$ gives non-trivial factors!

From L3.3: *"If these factors will divide z+1 and z-1, then if I take the greatest common divisors between N and z+1 and between N and z-1, factors p and q will be contained in those."*

**The Quantum Advantage:**

| Approach | Period Finding Complexity |
|----------|--------------------------|
| Classical (best known) | $O(\exp(\sqrt[3]{n \log n}))$ |
| Quantum (Shor's) | $O(n^2 \log n \log \log n)$ |

This exponential speedup is what makes Shor's algorithm a threat to RSA encryption!

In [ ]:
def demonstrate_period_to_factor(N: int, a: int):
    """
    Demonstrate how finding the period r leads to factors of N.
    
    From L3.3: "If you can find the period r, you can substitute that period 
    here and get your non-trivial solution and hence do the factorization."
    """
    from math import gcd
    
    print("Period-to-Factor Conversion Demonstration")
    print("=" * 70)
    print(f"Goal: Factor N = {N}")
    print(f"Chosen base: a = {a}")
    print()
    
    # Step 1: Find the period of f(x) = a^x mod N
    print("Step 1: Find period r of f(x) = a^x mod N")
    print("-" * 70)
    
    # Compute f(x) for x = 0, 1, 2, ...
    values = []
    x = 0
    seen = {}
    
    while True:
        fx = pow(a, x, N)  # a^x mod N
        values.append((x, fx))
        
        if fx in seen:
            r = x - seen[fx]
            break
        seen[fx] = x
        x += 1
        
        if x > 100:  # Safety limit
            print("Period not found within 100 iterations")
            return
    
    print(f"  f(x) = {a}^x mod {N}:")
    for i, (xi, fxi) in enumerate(values[:r+3]):
        marker = " ← period starts repeating" if i == r else ""
        print(f"    f({xi}) = {a}^{xi} mod {N} = {fxi}{marker}")
    
    print(f"\n  Period found: r = {r}")
    print(f"  Verification: {a}^{r} mod {N} = {pow(a, r, N)} ✓" if pow(a, r, N) == 1 else "")
    print()
    
    # Step 2: Check if r is even
    print("Step 2: Check if r is even")
    print("-" * 70)
    
    if r % 2 == 0:
        print(f"  r = {r} is even ✓")
    else:
        print(f"  r = {r} is ODD ✗")
        print("  Need to try a different value of a")
        return
    print()
    
    # Step 3: Compute z = a^(r/2) mod N
    print("Step 3: Compute z = a^(r/2) mod N")
    print("-" * 70)
    
    z = pow(a, r // 2, N)
    print(f"  z = a^(r/2) mod N = {a}^{r//2} mod {N} = {z}")
    print()
    
    # Step 4: Check non-triviality
    print("Step 4: Check non-triviality (z ≠ ±1 mod N)")
    print("-" * 70)
    
    if z == 1:
        print(f"  z = {z} ≡ 1 (mod {N}) ✗")
        print("  Trivial solution - need different a")
        return
    elif z == N - 1:
        print(f"  z = {z} ≡ -1 (mod {N}) ✗")
        print("  Trivial solution - need different a")
        return
    else:
        print(f"  z = {z} ≢ ±1 (mod {N}) ✓")
    print()
    
    # Step 5: Compute factors using GCD
    print("Step 5: Extract factors using GCD")
    print("-" * 70)
    
    z_plus_1 = z + 1
    z_minus_1 = z - 1
    
    factor1 = gcd(z_plus_1, N)
    factor2 = gcd(z_minus_1, N)
    
    print(f"  z + 1 = {z_plus_1}")
    print(f"  z - 1 = {z_minus_1}")
    print()
    print(f"  gcd(z+1, N) = gcd({z_plus_1}, {N}) = {factor1}")
    print(f"  gcd(z-1, N) = gcd({z_minus_1}, {N}) = {factor2}")
    print()
    
    # Verify
    factors = sorted([f for f in [factor1, factor2] if f not in [1, N]])
    
    if factors:
        print(f"🎉 Non-trivial factors found: {factors}")
        print(f"   Verification: {factors[0]} × {N // factors[0]} = {factors[0] * (N // factors[0])} = N ✓")
    else:
        print("No non-trivial factors found with this a")
    
    return r, factors


# Demonstrate with N=15, a=7
r, factors = demonstrate_period_to_factor(N=15, a=7)

### 5.5.2 State Evolution Through Shor's Circuit

The quantum part of Shor's algorithm uses QPE to find the period:

| Stage | Counting Register | Work Register | Key Insight |
|-------|------------------|---------------|-------------|
| **Initial** | $\|0\rangle^{\otimes t}$ | $\|1\rangle$ | Start with $\|1\rangle$ for modular exponentiation |
| **After H⊗t** | $\frac{1}{\sqrt{M}}\sum_{x=0}^{M-1}\|x\rangle$ | $\|1\rangle$ | $M = 2^t$ superposition states |
| **After C-U^(2^j)** | (phase kickback) | $\|a^x \mod N\rangle$ | Controlled modular exponentiation |
| **After Oracle** | $\frac{1}{\sqrt{M}}\sum_{x=0}^{M-1}e^{2\pi i sx/r}\|x\rangle$ | (entangled) | State encodes period information |
| **After QFT†** | $\|m\rangle$ where $m \approx sM/r$ | — | Peaks at multiples of $M/r$ |

**The Key State After the Oracle:**

From L3.4: *"I can club together all the red type of states... the superposition of everything at locations that are r apart."*

$$|\psi\rangle = \frac{1}{\sqrt{M}}\sum_{x=0}^{M-1}|x\rangle|a^x \mod N\rangle$$

The periodic structure of $a^x \mod N$ (period $r$) creates peaks at $m = 0, M/r, 2M/r, ...$

In [ ]:
def trace_shor_evolution(a: int, N: int, n_count: int = 4, verbose: bool = True) -> dict:
    """
    Trace the quantum state evolution through Shor's algorithm.
    
    From L3.4: "Let's take a look at how the qubit states are evolving 
    as they're going through these different steps."
    
    Args:
        a: Base for modular exponentiation
        N: Number to factor
        n_count: Number of counting qubits
        verbose: Print detailed output
    """
    M = 2 ** n_count  # Number of superposition states
    stages = {}
    
    # Find the actual period
    r = 1
    while pow(a, r, N) != 1:
        r += 1
        if r > N:
            raise ValueError(f"Period not found for a={a}, N={N}")
    
    if verbose:
        print("Shor's Algorithm State Evolution")
        print("=" * 70)
        print(f"Parameters: a={a}, N={N}, t={n_count} counting qubits")
        print(f"M = 2^t = {M}")
        print(f"Actual period: r = {r}")
        print()
    
    # Stage 1: After Hadamards
    if verbose:
        print("Stage 1 - After Hadamards on counting register:")
        print("-" * 70)
        print(f"  |ψ⟩ = (1/√{M}) Σ_{{x=0}}^{{{M-1}}} |x⟩ ⊗ |1⟩")
        print()
    
    # Stage 2: After modular exponentiation oracle
    if verbose:
        print("Stage 2 - After modular exponentiation oracle:")
        print("-" * 70)
        print(f"  U|x⟩|y⟩ = |x⟩|y · a^x mod N⟩")
        print(f"  Starting from |1⟩:")
        print(f"  |ψ⟩ = (1/√{M}) Σ_{{x=0}}^{{{M-1}}} |x⟩ |{a}^x mod {N}⟩")
        print()
        print("  First few values of a^x mod N:")
        for x in range(min(r + 2, 8)):
            ax_mod_N = pow(a, x, N)
            marker = " ← period!" if x == r else ""
            print(f"    x={x}: {a}^{x} mod {N} = {ax_mod_N}{marker}")
        print(f"  (Pattern repeats with period r = {r})")
        print()
    
    # Stage 3: Grouping by function value
    if verbose:
        print("Stage 3 - Grouping by function output (key insight!):")
        print("-" * 70)
        print("  States with same f(x) = a^x mod N are grouped together:")
        print()
    
    # Group x values by their function output
    groups = {}
    for x in range(M):
        fx = pow(a, x, N)
        if fx not in groups:
            groups[fx] = []
        groups[fx].append(x)
    
    if verbose:
        for fx, x_vals in sorted(groups.items())[:4]:  # Show first 4 groups
            x_str = ", ".join(str(x) for x in x_vals[:5])
            if len(x_vals) > 5:
                x_str += ", ..."
            print(f"    f(x) = {fx}: x ∈ {{{x_str}}}")
            print(f"         Note: x values differ by r = {r}")
        print()
    
    # Stage 4: State before inverse QFT
    if verbose:
        print("Stage 4 - State structure before inverse QFT:")
        print("-" * 70)
        print(f"  The x values in each group are: x₀, x₀+r, x₀+2r, ...")
        print(f"  This periodic spacing (period r) creates peaks in QFT!")
        print()
        print("  Inverse QFT converts this periodicity into peaks at:")
        print(f"    m ≈ s·M/r for s = 0, 1, 2, ..., r-1")
        print()
        
        expected_peaks = [s * M // r for s in range(r)]
        print(f"  Expected peaks (M={M}, r={r}):")
        for s in range(min(r, 6)):
            m = s * M / r
            phase = s / r
            print(f"    s={s}: m ≈ {m:.1f} → phase = s/r = {s}/{r} = {phase:.4f}")
    
    # Stage 5: Measurement and classical post-processing
    if verbose:
        print()
        print("Stage 5 - After measurement and continued fractions:")
        print("-" * 70)
        print(f"  Measured m gives phase estimate θ ≈ s/r")
        print(f"  Continued fractions extract r from θ")
        print()
        print("  Example continued fraction expansion:")
        
        # Demonstrate for s=1
        s = 1
        theta = s / r
        print(f"    θ = {s}/{r} = {theta:.6f}")
        print(f"    Continued fraction → denominator = {r}")
        print(f"    Period r = {r} found!")
    
    stages['period'] = r
    stages['M'] = M
    stages['expected_peaks'] = [s * M / r for s in range(r)]
    
    return stages


# Trace evolution for N=15, a=7
stages = trace_shor_evolution(a=7, N=15, n_count=4)

### 5.5.3 Why Continued Fractions?

**The Extraction Problem:**
From measurement we get $m$ (an integer), but we need to find $r$.

The relationship is:
$$\frac{m}{M} \approx \frac{s}{r}$$

where $s$ is some integer $< r$, and $M = 2^t$.

**Continued Fractions to the Rescue:**
Given a decimal approximation, continued fractions find the **best rational approximation** with a small denominator.

**Algorithm:**
1. Measure integer $m$
2. Compute $\theta = m/M$ (decimal approximation)
3. Apply continued fraction expansion to $\theta$
4. The denominator of a convergent gives candidate for $r$
5. Verify: check if $a^r \equiv 1 \pmod{N}$

In [ ]:
from fractions import Fraction

def explore_continued_fractions(m: int, M: int, N: int, a: int):
    """
    Explore how continued fractions extract the period from measurement.
    
    From L3.4: "The idea is that continued fractions is a really nice 
    classical algorithm for extracting ratios of two small integers."
    """
    print("Continued Fractions for Period Extraction")
    print("=" * 60)
    print(f"Measured value: m = {m}")
    print(f"Total states: M = {M}")
    print(f"Phase estimate: θ = m/M = {m}/{M} = {m/M:.10f}")
    print()
    
    # Use Fraction to find best rational approximation
    theta = Fraction(m, M)
    
    print("Continued Fraction Convergents:")
    print("-" * 60)
    
    # Compute continued fraction expansion manually
    x = Fraction(m, M)
    convergents = []
    cf_coeffs = []
    
    for i in range(10):  # Max iterations
        if x.denominator == 0:
            break
        
        a_i = int(x)  # Integer part
        cf_coeffs.append(a_i)
        
        # Compute convergent from coefficients
        if len(cf_coeffs) == 1:
            p, q = a_i, 1
        else:
            # Compute convergent using recurrence
            p_prev, p_prev2 = convergents[-1][0], (1 if len(convergents) == 1 else convergents[-2][0])
            q_prev, q_prev2 = convergents[-1][1], (0 if len(convergents) == 1 else convergents[-2][1])
            p = a_i * p_prev + (p_prev2 if len(convergents) > 1 else 0)
            q = a_i * q_prev + (q_prev2 if len(convergents) > 1 else 1)
        
        convergents.append((p, q) if len(convergents) == 0 else (p, q))
        
        # Check if this denominator is our period
        if q <= N:
            check = pow(a, q, N)
            is_period = "✓ Valid period!" if check == 1 else f"a^{q} mod N = {check}"
        else:
            is_period = "q > N, too large"
        
        print(f"  [{', '.join(str(c) for c in cf_coeffs)}] → {p}/{q} ≈ {p/q:.10f}  {is_period}")
        
        # Compute fractional part for next iteration
        frac_part = x - a_i
        if frac_part == 0:
            break
        x = Fraction(1, frac_part)
    
    # Also use Python's limit_denominator for comparison
    print()
    print("Using Fraction.limit_denominator (simpler approach):")
    print("-" * 60)
    for max_denom in [4, 8, 16, N]:
        approx = Fraction(m, M).limit_denominator(max_denom)
        check = pow(a, approx.denominator, N) if approx.denominator <= N else -1
        validity = "✓" if check == 1 else "✗"
        print(f"  limit={max_denom:3d}: θ ≈ {approx} = {float(approx):.10f}  "
              f"r_candidate = {approx.denominator} {validity}")


# Example: For N=15, a=7, the period is r=4
# A measurement of m=4 (when M=16) should give phase 4/16 = 1/4
explore_continued_fractions(m=4, M=16, N=15, a=7)

### 5.5.4 Interactive: The Complete Shor Pipeline

Let's trace the complete factorization pipeline from measurement to factors.

**The Full Classical Post-Processing:**
1. **Measure** → get integer $m$
2. **Phase estimation** → $\theta = m/M \approx s/r$
3. **Continued fractions** → extract candidate $r$
4. **Verify period** → check $a^r \equiv 1 \pmod{N}$
5. **Factor extraction** → compute $\gcd(a^{r/2} \pm 1, N)$

In [ ]:
def complete_shor_pipeline(N: int, a: int, n_count: int = 4, simulate_measurement: int = None):
    """
    Complete Shor's algorithm pipeline: measurement → period → factors.
    
    From L3.3: "If z squared is 1 mod N, that means z+1 times z-1 
    is a multiple of N. So gcd of z+1 with N and gcd of z-1 with N
    should give us factors."
    """
    from math import gcd
    from fractions import Fraction
    
    M = 2 ** n_count
    
    print("Complete Shor's Algorithm Pipeline")
    print("=" * 70)
    print(f"Goal: Factor N = {N} using base a = {a}")
    print(f"Counting qubits: t = {n_count}, M = 2^t = {M}")
    print()
    
    # Find actual period for simulation
    r_actual = 1
    while pow(a, r_actual, N) != 1:
        r_actual += 1
    print(f"[Hidden: actual period r = {r_actual}]")
    print()
    
    # Simulate measurement (would come from quantum circuit)
    if simulate_measurement is None:
        # Simulate measuring at one of the expected peaks
        import random
        s = random.randint(1, r_actual - 1)  # Avoid s=0
        m = round(s * M / r_actual)
    else:
        m = simulate_measurement
    
    print("STEP 1: Quantum Measurement")
    print("-" * 70)
    print(f"  Measured value: m = {m}")
    print()
    
    print("STEP 2: Phase Estimation")
    print("-" * 70)
    theta = m / M
    print(f"  θ = m/M = {m}/{M} = {theta:.10f}")
    print(f"  This should approximate some s/r")
    print()
    
    print("STEP 3: Continued Fractions")
    print("-" * 70)
    frac = Fraction(m, M).limit_denominator(N)
    r_candidate = frac.denominator
    print(f"  Best rational approximation: {frac}")
    print(f"  Candidate period: r = {r_candidate}")
    print()
    
    print("STEP 4: Period Verification")
    print("-" * 70)
    check = pow(a, r_candidate, N)
    if check == 1:
        print(f"  ✓ Verified: {a}^{r_candidate} mod {N} = 1")
        r = r_candidate
    else:
        print(f"  ✗ Failed: {a}^{r_candidate} mod {N} = {check}")
        # Try multiples
        for mult in range(2, 5):
            r_try = r_candidate * mult
            if pow(a, r_try, N) == 1:
                print(f"  ✓ Found at multiple: r = {r_try}")
                r = r_try
                break
        else:
            print("  Need to repeat measurement")
            return None
    print()
    
    print("STEP 5: Factor Extraction (The Magic!)")
    print("-" * 70)
    
    if r % 2 != 0:
        print(f"  Period r = {r} is odd → need to retry with different 'a'")
        return None
    
    z = pow(a, r // 2, N)
    print(f"  z = a^(r/2) mod N = {a}^{r//2} mod {N} = {z}")
    print()
    
    if z == N - 1:  # z ≡ -1 (mod N)
        print(f"  z ≡ -1 (mod N) → trivial factors, need different 'a'")
        return None
    
    print(f"  Using: z² ≡ 1 (mod N) → (z-1)(z+1) ≡ 0 (mod N)")
    print()
    
    factor1 = gcd(z - 1, N)
    factor2 = gcd(z + 1, N)
    
    print(f"  gcd(z-1, N) = gcd({z-1}, {N}) = {factor1}")
    print(f"  gcd(z+1, N) = gcd({z+1}, {N}) = {factor2}")
    print()
    
    # Find non-trivial factors
    factors = []
    for f in [factor1, factor2]:
        if f != 1 and f != N:
            factors.append(f)
    
    if factors:
        print("RESULT:")
        print("=" * 70)
        print(f"  N = {N} = {factors[0]} × {N // factors[0]}")
        print(f"  ✓ Factorization successful!")
        return factors[0], N // factors[0]
    else:
        print("  No non-trivial factors found, retry needed")
        return None


# Run the complete pipeline for N=15
complete_shor_pipeline(N=15, a=7, n_count=4, simulate_measurement=4)

In [ ]:
# Try different values of N and a
print("=" * 70)
print("Try factoring N = 21 with a = 2:")
print("=" * 70)
complete_shor_pipeline(N=21, a=2, n_count=5, simulate_measurement=5)

### 5.5.5 Key Takeaways: State Evolution Summary

| Aspect | Shor's Algorithm |
|--------|------------------|
| **Core Identity** | $a^r \equiv 1 \pmod{N} \Rightarrow (a^{r/2}-1)(a^{r/2}+1) \equiv 0 \pmod{N}$ |
| **Problem Reduction** | Factoring → Period Finding |
| **Quantum Subroutine** | QPE on modular exponentiation unitary $U\|y\rangle = \|ay \bmod N\rangle$ |
| **State After Oracle** | $\frac{1}{\sqrt{M}}\sum_{x=0}^{M-1}\|x\rangle\|a^x \bmod N\rangle$ |
| **Key Insight** | States with same $a^x \bmod N$ group together with spacing $r$ |
| **QFT Output** | Peaks at $m \approx sM/r$ for $s = 0, 1, ..., r-1$ |
| **Classical Post-Processing** | Continued fractions extract $r$ from $m/M \approx s/r$ |
| **Factor Extraction** | $\gcd(a^{r/2} \pm 1, N)$ gives non-trivial factors |
| **Success Probability** | $\geq 1/2$ per run (high probability of usable $r$) |
| **Complexity** | $O(n^2 \log n \log \log n)$ vs $O(\exp(n^{1/3}))$ classical |

**Why It Works:**
1. Modular exponentiation creates **periodic structure** in the quantum state
2. Inverse QFT converts this periodicity into **measurable peaks**
3. Continued fractions efficiently extract the **exact period** from approximate phase
4. Number theory converts period to factors via **GCD computation**

**Critical Requirements:**
- $\gcd(a, N) = 1$ (otherwise we found a factor trivially!)
- $r$ must be even (probability $\geq 1/2$)
- $a^{r/2} \not\equiv -1 \pmod{N}$ (probability $\geq 1/2$)

**The Exponential Speedup:**
- Classical: Best known algorithm is $O(\exp(n^{1/3}))$ (number field sieve)
- Quantum: Shor's algorithm is $O(n^2 \log n \log \log n)$
- This is why RSA encryption is vulnerable to quantum computers!

## Section 6: Running Shor's Algorithm

In [ ]:
def run_shor(a: int, N: int, n_count: int, shots: int = 2048) -> Dict:
    """
    Run Shor's algorithm and analyze results.
    
    Returns:
        Dictionary with measurement results and analysis
    """
    # Build and run circuit
    qc = shor_circuit(a, N, n_count)
    
    # Decompose the circuit to basic gates before simulation
    qc_decomposed = qc.decompose().decompose().decompose()
    
    simulator = AerSimulator()
    result = simulator.run(qc_decomposed, shots=shots).result()
    counts = result.get_counts()
    
    # Analyze each measurement outcome
    analysis = []
    for bitstring, count in counts.items():
        # Convert to integer (Qiskit returns LSB first)
        m = int(bitstring[::-1], 2)
        
        # Get period candidates from continued fractions
        candidates = extract_period_candidates(m, n_count, N)
        
        # Find valid periods
        valid_periods = [r for r in candidates if r > 0 and verify_period(a, r, N)]
        
        analysis.append({
            'bitstring': bitstring,
            'measurement': m,
            'count': count,
            'probability': count / shots,
            'phase_estimate': m / (2**n_count),
            'candidates': candidates,
            'valid_periods': valid_periods
        })
    
    # Sort by count
    analysis.sort(key=lambda x: -x['count'])
    
    return {
        'counts': counts,
        'analysis': analysis,
        'n_count': n_count,
        'a': a,
        'N': N
    }


# Run Shor's algorithm
results = run_shor(a=7, N=15, n_count=8, shots=4096)

print("Shor's Algorithm Results")
print("=" * 70)
print(f"N = {results['N']}, a = {results['a']}")
print(f"\nExpected period r = 4 (since 7^4 ≡ 1 mod 15)")
print(f"Expected phases: 0/4, 1/4, 2/4, 3/4 = 0, 0.25, 0.5, 0.75")
print(f"\n{'m':>6} | {'Phase':>8} | {'Prob':>6} | {'Candidates':>15} | Valid Periods")
print("-" * 70)

for item in results['analysis'][:10]:  # Top 10 results
    m = item['measurement']
    phase = item['phase_estimate']
    prob = item['probability']
    cands = str(item['candidates'][:4])  # First 4 candidates
    valid = item['valid_periods']
    
    print(f"{m:>6} | {phase:>8.4f} | {prob:>6.3f} | {cands:>15} | {valid}")

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Histogram of measurement outcomes
ax1 = axes[0]
measurements = [item['measurement'] for item in results['analysis']]
counts_list = [item['count'] for item in results['analysis']]

# Group by approximate phase
phases = [m / 256 for m in measurements]  # 256 = 2^8
expected_phases = [0, 0.25, 0.5, 0.75]

ax1.bar(measurements, counts_list, width=3, alpha=0.7, color='steelblue')

# Mark expected phases
for p in expected_phases:
    m_expected = int(p * 256)
    ax1.axvline(x=m_expected, color='red', linestyle='--', alpha=0.7)
    ax1.annotate(f's/r={p:.2f}', (m_expected, max(counts_list)*0.9),
                 rotation=90, fontsize=8, color='red')

ax1.set_xlabel('Measurement m')
ax1.set_ylabel('Counts')
ax1.set_title("Shor's Algorithm: Phase Estimation Results")

# Right: Phase distribution
ax2 = axes[1]

# Bin measurements into phase ranges
n_bins = 32
phase_counts = {}
for item in results['analysis']:
    phase_bin = round(item['phase_estimate'] * n_bins) / n_bins
    phase_counts[phase_bin] = phase_counts.get(phase_bin, 0) + item['count']

phases = list(phase_counts.keys())
counts = list(phase_counts.values())

ax2.bar(phases, counts, width=0.025, alpha=0.7, color='steelblue')

for p in expected_phases:
    ax2.axvline(x=p, color='red', linestyle='--', linewidth=2, alpha=0.7)

ax2.set_xlabel('Phase θ = s/r')
ax2.set_ylabel('Counts')
ax2.set_title('Phase Distribution (peaks at s/r for s = 0,1,2,3)')
ax2.set_xticks([0, 0.25, 0.5, 0.75, 1.0])

plt.tight_layout()
plt.show()

print("\n💡 The peaks occur at phases 0, 1/4, 2/4, 3/4 = s/4 for s=0,1,2,3")
print("   This tells us the period r = 4!")

## Section 7: Complete Factorization Pipeline

In [ ]:
def shor_factor(N: int, max_attempts: int = 10, verbose: bool = True) -> Tuple[int, int]:
    """
    Complete Shor's algorithm to factor N.
    
    Currently only supports N=15.
    
    Args:
        N: Number to factor
        max_attempts: Maximum random a values to try
        verbose: Print progress
    
    Returns:
        Tuple of factors (p, q)
    """
    if verbose:
        print(f"Factoring N = {N}")
        print("=" * 50)
    
    # Classical preprocessing
    is_simple, factor = classical_preprocessing(N)
    if is_simple:
        if verbose:
            print(f"Classical preprocessing found factor: {factor}")
        return factor, N // factor
    
    n_count = 8  # Good precision for N=15
    
    for attempt in range(max_attempts):
        if verbose:
            print(f"\nAttempt {attempt + 1}:")
        
        # Pick random a
        a, lucky_factor = pick_random_a(N)
        if verbose:
            print(f"  Chose a = {a}")
        
        if lucky_factor:
            if verbose:
                print(f"  Lucky! gcd({a}, {N}) = {lucky_factor}")
            return lucky_factor, N // lucky_factor
        
        # Run quantum order finding
        if verbose:
            print(f"  Running quantum order finding...")
        
        results = run_shor(a, N, n_count, shots=2048)
        
        # Try to extract period from measurements
        for item in results['analysis'][:5]:  # Check top 5 measurements
            for r in item['valid_periods']:
                if r > 1:  # Skip trivial period
                    if verbose:
                        print(f"  Found period r = {r}")
                    
                    # Try to extract factors
                    f1, f2 = factors_from_period(a, r, N)
                    
                    if f1 and f2:
                        if verbose:
                            print(f"  ✓ Found factors: {N} = {f1} × {f2}")
                        return f1, f2
                    elif verbose:
                        print(f"  Period {r} didn't yield factors")
    
    raise ValueError(f"Failed to factor {N} after {max_attempts} attempts")


# Factor 15!
p, q = shor_factor(15, verbose=True)

print(f"\n" + "=" * 50)
print(f"🎉 SUCCESS: 15 = {p} × {q}")

## Section 8: Trap Demonstrations

In [ ]:
# TRAP 1: Insufficient precision in counting register
print("TRAP 1: Insufficient Counting Qubits")
print("=" * 50)

# Try with very few counting qubits
for n_count in [2, 4, 6, 8]:
    results = run_shor(a=7, N=15, n_count=n_count, shots=1024)
    
    # Count how many measurements give valid period
    valid_count = 0
    total_count = 0
    for item in results['analysis']:
        total_count += item['count']
        if 4 in item['valid_periods']:  # True period is 4
            valid_count += item['count']
    
    success_rate = valid_count / total_count
    print(f"n_count = {n_count}: Success rate = {success_rate:.1%}")

print("\n💡 More counting qubits → higher success rate")

In [ ]:
# TRAP 2: Bad choice of a
print("\nTRAP 2: Problematic Values of a")
print("=" * 50)

N = 15

# Test different values of a
for a in [2, 4, 7, 11, 14]:
    # Check if coprime
    d = gcd(a, N)
    
    if d > 1:
        print(f"a = {a}: gcd({a}, {N}) = {d} → Found factor directly!")
        continue
    
    # Find period classically
    r = 1
    while pow(a, r, N) != 1:
        r += 1
    
    # Check if usable
    if r % 2 != 0:
        print(f"a = {a}: period r = {r} (odd) → UNUSABLE")
    elif pow(a, r//2, N) == N - 1:
        print(f"a = {a}: period r = {r}, but a^(r/2) ≡ -1 → UNUSABLE")
    else:
        f1, f2 = factors_from_period(a, r, N)
        print(f"a = {a}: period r = {r} → factors {f1}, {f2} ✓")

## Section 9: Exercises

### Exercise 1: Different Base (Beginner)
Run Shor's algorithm with a = 2 for N = 15. What happens? Why?

In [ ]:
# TODO: Exercise 1
# Run shor_factor or run_shor with a=2, N=15
# Analyze why it fails or succeeds

# Your code here:

### Exercise 2: Success Probability (Intermediate)
Run 100 complete Shor factorizations with random a values. What fraction succeed on the first quantum measurement? How many require multiple measurements?

In [ ]:
# TODO: Exercise 2
# Your code here:

### Exercise 3: Continued Fractions Analysis (Advanced)
For measurement outcome m = 77 with 8 counting qubits, manually compute the continued fraction expansion and find all convergents. Which one gives the correct period?

In [ ]:
# TODO: Exercise 3
# Your code here:

## Section 10: Quick Knowledge Check

**Q1**: Why do we initialize the work register to |1⟩ instead of |0⟩?

<details>
<summary>Click for answer</summary>

The modular exponentiation computes $a^x \cdot y \mod N$. Starting with $y=1$ gives us $a^x \cdot 1 = a^x \mod N$, which is exactly what we want to find the period of. Starting with $y=0$ would always give 0.
</details>

**Q2**: What's the probability that a randomly chosen a gives usable period?

<details>
<summary>Click for answer</summary>

For N = pq (product of two distinct primes), the probability is at least 1/2. This is because:
- Probability that r is even: ≥ 1/2
- Given r is even, probability that $a^{r/2} \not\equiv -1$: ≥ 1/2
- Combined: ≥ 1/4, but correlations improve this to ≥ 1/2
</details>

**Q3**: Why doesn't Shor's algorithm break AES encryption?

<details>
<summary>Click for answer</summary>

Shor's algorithm solves:
1. Integer factorization
2. Discrete logarithm

AES is a symmetric block cipher - its security doesn't rely on either of these problems. Grover's algorithm gives a quadratic speedup for brute-force attacks on AES, but doubling the key length (AES-256) restores security.
</details>

## Section 11: Summary & Cryptographic Implications

### What We Learned

1. **Factoring reduces to period finding** via number theory
2. **QPE extracts periods** with polynomial efficiency
3. **Continued fractions** recover the period from noisy estimates
4. **Modular exponentiation** is the expensive quantum step

### Cryptographic Impact

| System | Basis | Status with Shor |
|--------|-------|------------------|
| RSA | Factoring | 💀 Broken |
| Diffie-Hellman | Discrete log | 💀 Broken |
| ECC | Elliptic curve DL | 💀 Broken |
| AES | Symmetric | ⚠️ Key doubling needed |
| Lattice-based | Hard lattice problems | ✅ Believed quantum-safe |

### Timeline to Threat

Current estimates for cracking RSA-2048:
- **Logical qubits needed**: ~4,000
- **Physical qubits** (with error correction): ~millions
- **Estimated timeline**: 10-20+ years

But "harvest now, decrypt later" attacks make this urgent today!

### Next Steps

- **[Module 7.7: Grover's Algorithm](Module-07-Grovers-Search.md)**: Quadratic speedup for search
- **[Module 7.11: HHL Algorithm](Module-11-HHL.md)**: Another QPE application
- **Post-Quantum Cryptography**: NIST standards (ML-KEM, ML-DSA)